### Lab Tasks
- Use the following data prepration steps. 
- Use ``train_test_split`` to split the dataset into train and test dataset. set ``random_state = 0``.
- Use ``StandardScaler`` to scale feature set X. 
- In this lab use ``random_state = 0`` whenever is aplicable. 

### Data prepration

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('auto.csv')

In [3]:
data = data[data != '?']

C:\Users\obaid\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [4]:
data['horsepower'] = data['horsepower'].astype(float)
df = data[['cylinders', 'horsepower']]
g = df.groupby('cylinders').median()

In [5]:
data_imputed = df.apply(lambda grp: grp.fillna(grp.median()))

In [6]:
data['horsepower'] = data_imputed['horsepower'].astype(float)
data.drop(['car name'], axis = 1, inplace = True)

In [7]:
y = data['mpg']
X = data.drop(['mpg'], axis = 1)

In [8]:
X.head(5)

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,8,350.0,165.0,3693,11.5,70,1
1,8,318.0,150.0,3436,11.0,70,1
2,8,304.0,150.0,3433,12.0,70,1
3,8,302.0,140.0,3449,10.5,70,1
4,8,429.0,198.0,4341,10.0,70,1


In [9]:
y.head(5)

0    15.0
1    18.0
2    16.0
3    17.0
4    15.0
Name: mpg, dtype: float64

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

x_train_org,x_test_org,y_train,y_test=train_test_split(X,y,random_state=0)  ## org stands for the very original
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train_org)  ### you can fit and transform together
X_test = scaler.transform(x_test_org) 

C:\Users\obaid\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\obaid\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\obaid\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [11]:
X_train

array([[ 0.325963  ,  0.06824197, -0.51920175, ...,  0.07281474,
         0.50821018, -0.71362033],
       [-0.84043473, -1.12026065, -1.40992403, ...,  1.41511686,
         0.50821018,  1.79460734],
       [-0.84043473, -0.51626751, -0.438227  , ...,  0.63210729,
         0.78364526, -0.71362033],
       ...,
       [-0.84043473, -0.75007131, -0.78911759, ..., -0.03904377,
        -0.86896522,  0.5404935 ],
       [ 0.325963  ,  0.55533321, -0.438227  , ..., -0.41190547,
        -1.41983538, -0.71362033],
       [-0.84043473, -0.72084583, -0.19530274, ...,  0.52024878,
        -0.31809506,  1.79460734]])

#### Question 1
Use the ``LocallyLinearEmbedding`` to reduce the dimensionality of this dataset to 4 (``random_state = 0``). Train a polynomial regression of degree 2 on the reduced dataset. What is the test score of this model? (2 significant digits)

In [17]:
random_state = 0
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=4,  random_state=0)

X_train_reduced = lle.fit_transform(X_train)

X_test_reduced= lle.transform(X_test)

In [18]:
print(X_train.shape)
print(X_train_reduced.shape)

(297, 7)
(297, 4)


In [19]:
#####  transform to polynomial features

from sklearn.preprocessing import PolynomialFeatures
n=2   ## we are using 2 degress, usually 2 or 3 are enough for polynomial features 
poly = PolynomialFeatures(n)
X_train_poly = poly.fit_transform(X_train_reduced)
X_test_poly = poly.transform(X_test_reduced)

### perform linear regression on the polynomial features 
from sklearn.linear_model import LinearRegression
lreg = LinearRegression()

lreg.fit(X_train_poly,y_train)
print("train score is:",lreg.score(X_train_poly,y_train))
print("test score is:",lreg.score(X_test_poly,y_test))

train score is: 0.6820011447611809
test score is: 0.6351372701269989


#### Question 2
Use the ``LocallyLinearEmbedding`` to reduce the dimensionality of this dataset to 4 (``random_state = 0``). Train a support vector machine with kernel ``poly`` on the reduced dataset with the following parameters: 
```Python
gamma = 0.1 
C = 1
epsilon=0.5
```
What is the mean squared error of the reduced train set? 

In [22]:
from sklearn.svm import SVR 
svr_poly  = SVR(gamma=0.1, C= 1,epsilon=0.5, kernel='poly',degree=2)
svr_poly.fit(X_train_reduced, y_train)

SVR(C=1, cache_size=200, coef0=0.0, degree=2, epsilon=0.5, gamma=0.1,
  kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [23]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_train, svr_poly.predict(X_train_reduced))

63.13440711538705